In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 39.2 MB/s eta 0:00:00


In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import mediapipe as mp
import time
from moviepy.editor import VideoFileClip
from google.colab import drive

In [ ]:
# Монтирование Google Диска
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Создание директории для сохранения обрезанных кадров
output_folder = '/content/output_frames'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Инициализация объекта для обнаружения позы
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
# Загрузка видео
video_path = "/content/drive/MyDrive/Караеэномичи/right/gedan_barai_right/gedan_barai_right_front_view/Andrey_gedan_barai_front_view.mp4"
cap = cv2.VideoCapture(video_path)
#cap = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (640, 480), isColor=True)

In [ ]:
# Проверка успешности загрузки видео
if not cap.isOpened():
    print("Ошибка при загрузке видео")
    exit()

In [ ]:
# Пустой список для обрезанных кадров
cropped_frames = []

In [ ]:
'''# Обработка каждого кадра видео
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break'''

# Чтение первых 100 кадров из видео
frame_count = 0
start_time = time.time()
while frame_count < 25:
    ret, frame = cap.read()
    if not ret:
        break

    # Преобразование кадра в RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Обнаружение позы на кадре
    results = pose.process(frame_rgb)

    # Получение координат позы человека
    if results.pose_landmarks:
        pose_landmarks = results.pose_landmarks.landmark

        # Получение координат ключевых точек позы
        keypoints = [[lm.x * frame.shape[1], lm.y * frame.shape[0]] for lm in pose_landmarks]

        # Нахождение крайних точек позы
        min_x = int(min(keypoints, key=lambda x: x[0])[0]) - 50  # Увеличение области обрезки
        max_x = int(max(keypoints, key=lambda x: x[0])[0]) + 50  # Увеличение области обрезки
        min_y = int(min(keypoints, key=lambda x: x[1])[1]) - 50  # Увеличение области обрезки
        max_y = int(max(keypoints, key=lambda x: x[1])[1]) + 50  # Увеличение области обрезки

        # Проверка границ области обрезки
        min_x = max(min_x, 0)
        max_x = min(max_x, frame.shape[1])
        min_y = max(min_y, 0)
        max_y = min(max_y, frame.shape[0])

        # Обрезка кадра вокруг позы человека
        #cropped_frame = frame[min_y:max_y, min_x:max_x]

        # Находим текущие ширину и высоту обрезанного кадра:
        cropped_height = max_y - min_y
        cropped_width = max_x - min_x

        # Вычислите максимальное значение из `cropped_height` и `cropped_width`:
        max_cropped_size = max(cropped_height, cropped_width)

        # Вычислите разницу между `max_cropped_size` и `cropped_height`/`cropped_width`:
        diff_height = max_cropped_size - cropped_height
        diff_width = max_cropped_size - cropped_width

        # Измените границы области обрезки, добавив половину разницы к каждой стороне:
        min_x -= diff_width // 2
        max_x += diff_width // 2
        min_y -= diff_height // 2
        max_y += diff_height // 2

        # Проверяем границы области обрезки, чтобы они не выходили за пределы кадра:
        min_x = max(min_x, 0)
        max_x = min(max_x, frame.shape[1])
        min_y = max(min_y, 0)
        max_y = min(max_y, frame.shape[0])

        # Обрезаем кадр с использованием обновленных границ:
        cropped_frame = frame[min_y:max_y, min_x:max_x]

        # Определение размера текущего обрезанного кадра
        cropped_height, cropped_width, _ = cropped_frame.shape

        # Нахождение самого большого кадра
        if cropped_height * cropped_width > max_cropped_size:
           max_cropped_size = cropped_height * cropped_width
           max_cropped_frame = cropped_frame

    for frame in cropped_frames:
            cropped_height, cropped_width, _ = frame.shape[:3]
    if cropped_height * cropped_width < max_cropped_size:
            min_cropped_size = cropped_height * cropped_width
            min_cropped_frame = frame

        # Добавление обрезанного кадра в список cropped_frames
    cropped_frames.append(cropped_frame)

    # Закрытие окна обрезанного кадра
    cv2.destroyAllWindows()

    # Изменение размера остальных кадров на больше
    for i, frame in enumerate(cropped_frames):
        if frame.shape != max_cropped_frame.shape:
           resized_frame = cv2.resize(frame, (max_cropped_frame.shape[1], max_cropped_frame.shape[0]), interpolation=cv2.INTER_LINEAR_EXACT)
           cropped_frames[i] = resized_frame

    # Дорисовка контура на кадрах, которые меньше по размеру
    for i, frame in enumerate(cropped_frames):
        if frame.shape[0] < max_cropped_frame.shape[0] or frame.shape[1] < max_cropped_frame.shape[1]:
           contour = np.array([[0, 0], [frame.shape[1], 0], [frame.shape[1], frame.shape[0]], [0, frame.shape[0]]])
           cv2.drawContours(frame, [contour], -1, (0, 0, 0), thickness=2)
           cropped_frames[i] = frame

        # Отображение обрезанного кадра
        cv2_imshow(cropped_frame)

    # Вывод номера кадра
    print("Номер кадра:", frame_count)

    # Время обработки каждого кадра
    frame_time = time.time() - start_time
    print("Время обработки кадра:", frame_time)

    frame_count += 1

Номер кадра: 0
Время обработки кадра: 0.23311328887939453
Номер кадра: 1
Время обработки кадра: 0.3198988437652588
Номер кадра: 2
Время обработки кадра: 0.3957366943359375
Номер кадра: 3
Время обработки кадра: 0.46939969062805176
Номер кадра: 4
Время обработки кадра: 0.5487794876098633
Номер кадра: 5
Время обработки кадра: 0.6275973320007324
Номер кадра: 6
Время обработки кадра: 0.715383768081665
Номер кадра: 7
Время обработки кадра: 0.8017120361328125
Номер кадра: 8
Время обработки кадра: 0.8962078094482422
Номер кадра: 9
Время обработки кадра: 0.9563548564910889
Номер кадра: 10
Время обработки кадра: 1.0140085220336914
Номер кадра: 11
Время обработки кадра: 1.12125563621521
Номер кадра: 12
Время обработки кадра: 1.191826581954956
Номер кадра: 13
Время обработки кадра: 1.252460241317749
Номер кадра: 14
Время обработки кадра: 1.3847763538360596
Номер кадра: 15
Время обработки кадра: 1.5130188465118408
Номер кадра: 16
Время обработки кадра: 1.6624927520751953
Номер кадра: 17
Время обраб

In [ ]:
# Общее время обработки
total_time = time.time() - start_time
print("Общее время обработки:", total_time)

Общее время обработки: 27.805639505386353


In [ ]:
# Создание директории для сохранения кадров (если она не существует)
output_directory = "/content/cropped_frames"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Сохранение каждого кадра в директории
for i, frame in enumerate(cropped_frames):
    frame_path = os.path.join(output_directory, f"frame_{i}.jpg")  # Путь кадра
    # меняем размер кадра до 120х120пх
    resized_frame = cv2.resize(frame, (120, 120))

    #cv2.imwrite(frame_path, frame)

    # Сохраните измененный кадр с нужным качеством
    cv2.imwrite(frame_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])

print("Кадры сохранены в директории:", output_directory)


Кадры сохранены в директории: /content/cropped_frames


In [ ]:
# Путь к директории с кадрами
frames_directory = "/content/cropped_frames"

# Получение списка файлов кадров
frame_files = sorted(os.listdir(frames_directory))

# Получение первого кадра для определения размеров видео
first_frame_path = os.path.join(frames_directory, frame_files[0])
first_frame = cv2.imread(first_frame_path)
height, width, _ = first_frame.shape

# Задание новых размеров видео
new_width, new_height = 100, 100

# Путь и имя видеофайла для сохранения
output_video_path = "/content/output_video.mp4"

# Создание объекта для записи видео
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), 30.0, (width, height))

# Сохранение обработанного видео в хорошем качестве
#out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (640, 480))


# Запись каждого кадра в видео
for frame_file in frame_files:
    frame_path = os.path.join(frames_directory, frame_file)
    frame = cv2.imread(frame_path)
    out.write(frame)

# Закрытие записи видео
out.release()

# Отображение видео в Colab
'''video = Video(output_video_path, embed=True)
video'''


'video = Video(output_video_path, embed=True)\nvideo'

In [ ]:
# Добавление обрезанного кадра в список cropped_frames
cropped_frames.append(cropped_frame)


In [ ]:
# Запись обработанных кадров в видео
while True:
    ret, frame = cap.read()

    if not ret:
        break

In [ ]:
# Путь и имя видеофайла
output_video_path = "output_video_path"

In [ ]:
print("Видео сохранено по пути:", output_video_path)


Видео сохранено по пути: output_video_path
